In [1]:
import pandas as pd
df = pd.read_pickle('shared/Project-3_NYC_311_Calls.pkl')[['Created Date', 'Unique Key','Complaint Type']]

df['Created Date'] = pd.to_datetime(df['Created Date'])
df.set_index('Created Date', inplace=True)

In [2]:
df_2022 = df[df.index.year == 2022]
daily_complaints_2022 = df_2022.resample('D')['Unique Key'].count()
average_daily_complaints_2022 = daily_complaints_2022.mean()

print(f"Average number of daily complaints in 2022: {average_daily_complaints_2022}")

Average number of daily complaints in 2022: 8684.320547945206


In [3]:
daily_call_counts = df.resample('D').size()

max_calls_date = daily_call_counts.idxmax()
df_max_calls_day = df[df.index.date == max_calls_date.date()]
most_common_complaint = df_max_calls_day['Complaint Type'].mode()[0]

print(f"Date with the maximum number of calls: {max_calls_date}")
print(f"Most common complaint on this date: {most_common_complaint}")

Date with the maximum number of calls: 2020-08-04 00:00:00
Most common complaint on this date: Damaged Tree


In [5]:
import calendar

df['Month'] = df.index.month
monthly_call_counts = df.groupby('Month')['Unique Key'].count()

quietest_month = monthly_call_counts.idxmin()
quietest_month_count = monthly_call_counts.min()

quietest_month_name = calendar.month_name[quietest_month]

print(f"Quietest month: {quietest_month_name}")

Quietest month: December


In [6]:
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose

daily_df = df.resample('D').size()

decomposition = seasonal_decompose(daily_df, model='additive', period=365)

seasonal = decomposition.seasonal

decomposition_annual = seasonal_decompose(daily_df, model='additive', period=365)
decomposition_weekly = seasonal_decompose(daily_df, model='additive', period=7)

seasonal_annual = decomposition_annual.seasonal
seasonal_weekly = decomposition_weekly.seasonal

seasonal_value_annual = round(seasonal_annual[pd.to_datetime('2020-12-25')])
seasonal_value_weekly = round(seasonal_weekly[pd.to_datetime('2020-12-25')])

seasonal_value_annual, seasonal_value_weekly

(-1106, 183)

In [7]:
daily_df = df.resample('D').size()
autocorrelation_lag_1 = daily_df.autocorr(lag=1)

print(f"Autocorrelation with a lag of 1 day: {autocorrelation_lag_1}")

Autocorrelation with a lag of 1 day: 0.7517059728398577


In [8]:
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np

df_prophet = daily_df.reset_index()
df_prophet.columns = ['ds', 'y']

train = df_prophet.iloc[:-90]
test = df_prophet.iloc[-90:]

model = Prophet()
model.fit(train)

future = model.make_future_dataframe(periods=90)
forecast = model.predict(future)

y_pred = forecast['yhat'].iloc[-90:]

rmse = np.sqrt(mean_squared_error(test['y'], y_pred))
print(f"RMSE on the test set: {rmse}")

15:27:58 - cmdstanpy - INFO - Chain [1] start processing
15:27:59 - cmdstanpy - INFO - Chain [1] done processing


RMSE on the test set: 1231.513760758433
